# Dashboard de Ventas Japón años 2023
### 1. Preparación y limpieza inicial de los datos

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Sven-Bo/datasets/master/store_sales_2022-2023.csv"
dataframe = pd.read_csv(url)
dataframe.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount
0,1,52,1,CodeComet,Software Development Tools,Tokyo,1/1/2022,8,303.29
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,1/1/2022,8,173.53
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,1/2/2022,6,37.72
3,4,88,2,VarVista Pro,Educational Tools,Osaka,1/2/2022,6,10.47
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,1/3/2022,1,159.10


In [2]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          1730 non-null   int64  
 1   product_id        1730 non-null   int64  
 2   store_id          1730 non-null   int64  
 3   product_name      1730 non-null   object 
 4   product_category  1730 non-null   object 
 5   city              1730 non-null   object 
 6   date_of_sale      1730 non-null   object 
 7   quantity_sold     1730 non-null   int64  
 8   sales_amount      1730 non-null   float64
dtypes: float64(1), int64(4), object(4)
memory usage: 121.8+ KB


In [3]:
# --- CAMBIANDO EL TIPO DE DATO DE LA COLUMNA DATE OF SALE  Y CREANDO LAS COLUMNAS YEAR Y MONTH ---
dataframe = dataframe.assign(
    date_of_sale = lambda df: pd.to_datetime(df["date_of_sale"]),
    month = lambda df: df["date_of_sale"].dt.month,
    year = lambda df: df["date_of_sale"].dt.year
)

In [4]:
dataframe.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount,month,year
0,1,52,1,CodeComet,Software Development Tools,Tokyo,2022-01-01,8,303.29,1,2022
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,2022-01-01,8,173.53,1,2022
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,2022-01-02,6,37.72,1,2022
3,4,88,2,VarVista Pro,Educational Tools,Osaka,2022-01-02,6,10.47,1,2022
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,2022-01-03,1,159.10,1,2022


### 2. Calculando las ganancias por ciudad

In [5]:
YEAR = 2023
city_revenues = (
    # agrupando los datos por año y ciudad 
    dataframe.groupby(["city", "year"])["sales_amount"]
    # calculando ventas totales por grupo
    .sum()
    # re organizando la tabla para que sea mas agradable visualmente
    .unstack()
    # calculando el porcentaje de cambio entre 2022 y 2023
    .assign(change=lambda x: x.pct_change(axis=1)[YEAR] * 100)
    )
city_revenues

year,2022,2023,change
city,,,
Osaka,76914.92,81202.93,5.575004
Tokyo,79961.13,72717.66,-9.058739
Yokohama,63216.90,67110.89,6.159729


### 3. Calculando ganancias en ciudades individualmente

In [6]:
city = "Tokyo"
revenue = city_revenues.loc[city, YEAR]
change = city_revenues.loc[city, "change"]

f"Ganancias totales en {city}: ${revenue:,.2f}, Cambio de 2022 a 2023: {change:.2f}%"

'Ganancias totales en Tokyo: $72,717.66, Cambio de 2022 a 2023: -9.06%'

In [7]:
#dataframe.to_csv("Ventas Regionales Japón.csv")

### 4. Preparando los datos para las visualizaciones

In [11]:
# variables a filtrar
sel_city ="Tokyo"
viz_year = 2023

filtered_data = (
    dataframe.query("city == @sel_city & year == @viz_year")
    .groupby("month", dropna=False, as_index=False)["sales_amount"]
    .sum()
)

filtered_data

,month,sales_amount
0,1,7289.70
1,2,6281.59
2,3,6568.99
3,4,6307.56
4,5,5881.92
5,6,4662.89
6,7,6116.09
7,8,5553.45
8,9,5327.64
9,10,8650.56
